In [1]:
# LSTM
import pandas as pd
df = pd.read_csv('spam.csv',encoding = 'cp1252')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [2]:
df = df.iloc[::,[0,1]]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df = df.rename(columns = {'v1':'label' , 'v2':'message'})
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.message = df.message.str.lower()
df.head()

,label,message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [5]:
df.label.replace({'spam':0 , 'ham':1},inplace = True) 
# spam = 0
# Ham = 1

In [6]:
# In NLP I removed Stopwords and Punctuation but while building my embedding model no need to remove stopwords  , punctuation

In [7]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(df , test_size = .2)
train_X = train.iloc[::,1]
train_y = train.iloc[::,0]
test_X = test.iloc[::,1]
test_y = test.iloc[:: , 0]

In [8]:
test_y

4441    1
2117    1
98      1
3261    1
1105    1
       ..
2450    1
1118    1
4983    0
331     1
4152    0
Name: label, Length: 1115, dtype: int64

In [9]:
import numpy as np
import pandas as pd
from keras.layers import Dense , SimpleRNN , GRU , LSTM , Embedding
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [10]:
train_y = to_categorical(train_y) # one hot encoding happened (dummy variable categorization)
train_y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [11]:
max_num_words = 10000 # how many unique words I want to consider from entire corpus (full document)
seq_len = 50 # I will take 50-50 words of batches
embedding_size = 100

In [12]:
from keras.preprocessing.text import Tokenizer # it is used for tokenization (assign number or take to each word)
from keras_preprocessing.sequence import pad_sequences # to bring each embedding had a same length

In [13]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(df.message)

In [14]:
train_X = tokenizer.texts_to_sequences(train_X)
train_X = pad_sequences(train_X , maxlen=seq_len)

In [15]:
test_X = tokenizer.texts_to_sequences(test_X)
test_X = pad_sequences(test_X , maxlen=seq_len)

In [16]:
train_X[9]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          2, 1503,    7,  195,    5, 1080,  193,   89, 4927,  603, 4928,
         36,  620,  193,   50,   19,  141, 4929,  188, 4930,  671,   48,
       4931,   92,   36,    5,  338,   52])

In [17]:
test_X[9]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,   60,   87,    2, 1178, 8052])

In [18]:
model = Sequential() # initialize the network
model.add(Embedding(input_dim  = max_num_words , 
                    input_length = seq_len,
                    output_dim = embedding_size))

In [19]:
model.add(LSTM(5)) # 5 neurons
model.add(Dense(2 , activation = 'softmax')) # 2no. of classes ham and spam
adam = Adam(lr  =.001)
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [20]:
model.fit(train_X , train_y , epochs = 5,validation_split = .1)

Epoch 1/5
126/126 [==============================] - 7s 28ms/step - loss: 0.3034 - accuracy: 0.9167 - val_loss: 0.1273 - val_accuracy: 0.9910
Epoch 2/5
126/126 [==============================] - 2s 19ms/step - loss: 0.0976 - accuracy: 0.9868 - val_loss: 0.0700 - val_accuracy: 0.9910
Epoch 3/5
126/126 [==============================] - 2s 19ms/step - loss: 0.0483 - accuracy: 0.9960 - val_loss: 0.0505 - val_accuracy: 0.9933
Epoch 4/5
126/126 [==============================] - 2s 20ms/step - loss: 0.0250 - accuracy: 0.9980 - val_loss: 0.0460 - val_accuracy: 0.9910
Epoch 5/5
126/126 [==============================] - 3s 20ms/step - loss: 0.0143 - accuracy: 0.9993 - val_loss: 0.0392 - val_accuracy: 0.9933


In [21]:
pred_values = model.predict(test_X)
pred_values # ans in probabilitites

35/35 [==============================] - 1s 3ms/step


array([[0.00542333, 0.9945767 ],
       [0.00502718, 0.9949728 ],
       [0.00577277, 0.9942272 ],
       ...,
       [0.97763264, 0.02236737],
       [0.00368133, 0.9963187 ],
       [0.98395306, 0.01604694]], dtype=float32)

In [22]:
# to build the confusion matrix prob should be converted to 0 and 1
pred_in_classes = np.argmax(pred_values , axis = 1)
from sklearn.metrics import confusion_matrix , accuracy_score
confusion_matrix(test_y , pred_in_classes)

array([[142,  10],
       [  3, 960]], dtype=int64)

In [23]:
accuracy_score(test_y , pred_in_classes)

0.9883408071748879